In [31]:
import json

with open("../data/farm_details.json") as f:
    farm_data = json.load(f)

In [32]:
# remove the first line from each address

for farm in farm_data:
    # Get the full address from the farm data
    address = farm["info"]["address"]

    # Split the address by newline character and keep everything after the first line
    parts = address.split("\n")
    if len(parts) > 1:
        # Join the parts back, excluding the first line
        updated_address = "\n".join(parts[1:])
        farm["info"]["address"] = updated_address

## define base url for search

In [33]:
import requests

# Define base url for address search
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Set up search parameters: address, origins and type
parameters = {"searchText": "8400 Winterthur, Theaterstrasse 17",
              "origins": "address",
              "type": "locations",
             }

# Urllib.parse.urlencode turns parameters into url
# print(f"{base_url}{urllib.parse.urlencode(parameters)}")

In [34]:
import urllib
import pandas as pd

# Server request
r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

# Get data in json-format
data = json.loads(r.content)
data

# Take only the first server response, convert to data frame with relevant infos
df = pd.DataFrame.from_dict(list(data.values())[0][0], orient='columns')
df.iloc[[1,4,5,6,11,12],:1]

,attrs
featureId,2323240_0
label,Theaterstrasse 17 <b>8400 Winterthur</b>
lat,47.503517
lon,8.727852
x,262216.125
y,697128.5625


In [51]:
from typing import Dict


def get_coordinates(address: str, verbose: bool = False) -> Dict[float, float]:
    base_url = "https://api3.geo.admin.ch/rest/services/api/SearchServer?"
    parameters = {
        "searchText": address,
        "origins": "address",
        "type": "locations",
    }

    response = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")
    data = json.loads(response.content)

    if "results" in data and data["results"]:
        attrs = data["results"][0]["attrs"]
        if "lat" in attrs and "lon" in attrs:
            return {"lat": attrs["lat"], "lon": attrs["lon"]}
        else:
            print(f"Latitude and longitude not found in the results for address {address}") if verbose else None
    else:
        print(f"Unexpected data structure for address {address}: {data}") if verbose else None

    return None

In [52]:
valid_farms = []
amount_errors = 0

for farm in farm_data:
    address = farm['info']['address']
    coordinates = get_coordinates(address)
    if coordinates:
        farm['info']['coordinates'] = coordinates
        valid_farms.append(farm)
    else:
        amount_errors += 1

print(f"Amount of farms with errors: {amount_errors}")
print(f"In percentage: {amount_errors / len(farm_data) * 100}%")

Amount of farms with errors: 16
In percentage: 4.289544235924933%


In [53]:
farm_data = valid_farms

geojson = {
    "type": "FeatureCollection",
    "name": "buildings",
    "crs": {"type": "name", "properties": {"name": "urn:ogc:def:crs:EPSG::2056"}},
    "features": [],
}

# Convert each farm entry to a GeoJSON feature
for farm in farm_data:
    feature = {
        "type": "Feature",
        "properties": {
            "name": farm["name"],
            "address": farm["info"]["address"],
            "lebensmittel": farm["info"]["lebensmittel"],
            # more properties if needed
        },
        "geometry": {
            "type": "Point",
            "coordinates": [
                farm["info"]["coordinates"]["lon"],
                farm["info"]["coordinates"]["lat"],
            ],
        },
    }
    geojson["features"].append(feature)

{
    "type": "FeatureCollection",
    "name": "buildings",
    "crs": {
        "type": "name",
        "properties": {
            "name": "urn:ogc:def:crs:EPSG::2056"
        }
    },
    "features": [
        {
            "type": "Feature",
            "properties": {
                "name": "Ritzmann-Müller Betriebsgemeinschaft, 8459 Volken",
                "address": "Flaachtalstr. 43\n8459 Volken",
                "lebensmittel": [
                    {
                        "category": "Verkaufsstellen",
                        "products": []
                    },
                    {
                        "category": "Gemüse und andere pflanzliche Produkte",
                        "products": [
                            "Kartoffeln",
                            "Weisskohl",
                            "Zwiebeln"
                        ]
                    },
                    {
                        "category": "Getränke",
                        "products": [

In [55]:
with open('../data/farms.geojson', 'w', encoding='utf-8') as file:
    json.dump(geojson, file, ensure_ascii=False, indent=4)
